In [2]:
import os
import pandas as pd
from notebooks.utils import logger, latest_file, file_with_timestamp
from notebooks.config import FILTERED_DATASET_DIRECTORY, VALIDATED_DATASET_DIRECTORY
import re
from keras.models import load_model
import numpy as np
import tensorflow as tf

2024-05-09 15:49:31.344349: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 15:49:31.854091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 15:49:31.854170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 15:49:31.929399: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 15:49:32.066152: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dataset = pd.read_csv(latest_file(FILTERED_DATASET_DIRECTORY, 'filtered_dataset'), sep='\t', header=None, index_col=0)
model = load_model("models/bert_classifier.keras")

if not os.path.exists(VALIDATED_DATASET_DIRECTORY):
    os.makedirs(VALIDATED_DATASET_DIRECTORY)

BATCH_SIZE = 128

2024-04-08 10:55:07.140786: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 10:55:07.240976: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


KeyboardInterrupt: 

In [2]:
import numpy as np
from keras.applications import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from config import IMAGES_DIRECTORY
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os

# Load the ResNet50 model pre-trained on ImageNet data
model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

h = 0

img_path = os.path.join(IMAGES_DIRECTORY, "1077.jpg")
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(model.predict(x).shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
(1, 1536)


In [24]:
rows_to_delete = []
ids, hypothesis, premise = [], [], []

import random


def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)

for j in range(BATCH_SIZE * BATCH_SIZE):
    random_index = random.randint(0, len(dataset) - 1)
    row = dataset.iloc[random_index]
    index = row.name
    
    number = 0
    probabilities = row[5].split(',')
    while 0 < len(probabilities) > number and number < 5 and float(probabilities[number]) > 0.8:
        number += 1
    if number > 0:
        labels = row[3].split(',')[:number]
        text = ' '.join(row[3].split(',')[:number])
        caption = re.sub(r'[^\w\s]', '', row[1])
        ids.append(index)
        hypothesis.append(caption)
        premise.append(text)
        print(row[2])
        print(text)
        print(caption)
        print()
        if len(hypothesis) == 10:
            predictions = softmax(model.predict((hypothesis, premise)))
            print(predictions)
            for i, prediction in enumerate(predictions):
                if np.argmax(prediction) == 2:
                    rows_to_delete.append(ids[i])
            ids, hypothesis, premise = [], [], []
            break
    else:
        rows_to_delete.append(index)

https://media.istockphoto.com/photos/snail-on-the-tree-in-the-garden-fabulous-snail-picture-id587936538?k=6&m=587936538&s=612x612&w=0&h=WUJ2l8TmB6f5d6Q962IhduHoeEEQ8yK5HB-t3jFP1Q4=
snails and slugs snail lymnaeidae molluscs
snail on the tree in the garden 

https://centralamericaadventure2014.files.wordpress.com/2014/01/costa-rica-samara-first-sunset-on-the-beach.jpg
sky water transportation boat sea vehicle
first sunset on the beach

https://i.pinimg.com/736x/b9/5e/2d/b95e2df265ac06a2a0f3cce10fecea55--moon-rise-red-moon.jpg
moon water transportation sky celestial event full moon
cruises along the water as the moon rises over filming location 

https://photos.smugmug.com/North-America/Mexico/Mazunte-Beach-Oaxaca/i-x6cC2cj/0/2c2022a3/M/6888799278_3853311d90_o-M.jpg
sky horizon nature sunset sunrise
reward for waking up early for the boat ride

https://media-cdn.tripadvisor.com/media/photo-s/10/cf/52/11/the-pier-from-the-beach.jpg
pier beach sea ocean coast
the pier from the beach

http:

In [22]:
rows_to_delete = []
ids, hypothesis, premise = [], [], []

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)

for index, row in dataset.iterrows():
    number = 0
    probabilities = row[5].split(',')
    while 0 < len(probabilities) > number and number < 5 and float(probabilities[number]) > 0.8:
        number += 1
    if number > 0:
        labels = row[3].split(',')[:number]
        text = ' '.join(row[3].split(',')[:number])
        caption = re.sub(r'[^\w\s]', '', row[1])
        ids.append(index)
        hypothesis.append(caption)
        premise.append(text)
        if len(hypothesis) == BATCH_SIZE:
            predictions = softmax(model.predict((hypothesis, premise)))
            for i, prediction in enumerate(predictions):
                if np.argmax(prediction) == 2:
                    rows_to_delete.append(ids[i])
            ids, hypothesis, premise = [], [], []
    else:
        rows_to_delete.append(index)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step
4/4 ━━━━━━━━

In [29]:
DATA_FOLDER = os.path.abspath('../../data')
DATASET_DIRECTORY = os.path.join(DATA_FOLDER, 'dataset')
VALIDATED_DATASET_DIRECTORY = os.path.join(DATASET_DIRECTORY, 'validated_dataset')

validated_dataset = dataset.drop(rows_to_delete)
validated_dataset.to_csv(os.path.join(VALIDATED_DATASET_DIRECTORY, file_with_timestamp('validated_dataset.tsv')), sep='\t', header=False)

In [31]:
len(dataset)

342167

In [30]:
len(validated_dataset)

224282

In [1]:
!pip freeze > requirements.txt

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [3]:
!pip install tensorflow[and-cuda]

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 MB 1.1 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 5.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 845.8/845.8 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 MB 1.6 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
2024-05-09 16:07:29.896409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 16:07:29.897285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 16:07:29.931240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 16:07:31.707608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
!pip install tensorflow[and-cuda]

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
